# Nationality Predictor

# Step 1: Importing Libraries and Preparing the Dataset

In [ ]:
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.callbacks import EarlyStopping



In [2]:

# Load dataset and preprocess
def load_utkface(dataset_path):
    data = []
    labels = []
    for filename in os.listdir(dataset_path):
        if filename.endswith(".jpg"):
            parts = filename.split("_")
            if len(parts) == 4:
                age, gender, race, _ = parts
            elif len(parts) == 3:
                age, gender, race = parts
            else:
                continue
            try:
                age = int(age)
                gender = int(gender)
                race = int(race)
            except ValueError:
                continue
            if age < 10 or age > 60:
                continue
            img_path = os.path.join(dataset_path, filename)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (48, 48))
            data.append(img)
            labels.append((age, gender, race))
    return np.array(data), np.array(labels)

dataset_path = r"C:\Users\Addmin\Downloads\Nationality prediction\UTKFace"
X, y = load_utkface(dataset_path)



In [3]:
X = X / 255.0  # Normalize the images

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Separate labels into age, gender, and race
y_train_age = y_train[:, 0]
y_train_gender = y_train[:, 1]
y_train_race = y_train[:, 2]

y_test_age = y_test[:, 0]
y_test_gender = y_test[:, 1]
y_test_race = y_test[:, 2]

# Convert labels to categorical
y_train_gender = to_categorical(y_train_gender, num_classes=2)
y_train_race = to_categorical(y_train_race, num_classes=5)

y_test_gender = to_categorical(y_test_gender, num_classes=2)
y_test_race = to_categorical(y_test_race, num_classes=5)


# Step 2: Model Architecture

In [4]:
input_layer = Input(shape=(48, 48, 3))

# Convolutional layers
x = Conv2D(32, (3, 3), activation='relu')(input_layer)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)

# Output layers
age_output = Dense(1, name='age_output')(x)
gender_output = Dense(2, activation='softmax', name='gender_output')(x)
race_output = Dense(5, activation='softmax', name='race_output')(x)

# Create model
model = Model(inputs=input_layer, outputs=[age_output, gender_output, race_output])
model.compile(optimizer='adam', loss={'age_output': 'mse', 'gender_output': 'categorical_crossentropy', 'race_output': 'categorical_crossentropy'}, metrics={'age_output': 'mae', 'gender_output': 'accuracy', 'race_output': 'accuracy'})

# Summary of the model
model.summary()

# Save the model
model.save('nationality_prediction_model.h5')


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 48, 48, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 46, 46, 32)   896         ['input_1[0][0]']                
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 23, 23, 32)   0           ['conv2d[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 21, 21, 64)   18496       ['max_pooling2d[0][0]']          
                                                                                              

In [5]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(X_train, 
                    {'age_output': y_train_age, 'gender_output': y_train_gender, 'race_output': y_train_race},
                    validation_data=(X_test, {'age_output': y_test_age, 'gender_output': y_test_gender, 'race_output': y_test_race}),
                    epochs=50,
                    batch_size=32,
                    callbacks=[early_stopping])


Epoch 1/50
457/457 [==============================] - 83s 168ms/step - loss: 187.5372 - age_output_loss: 182.9638 - gender_output_loss: 1.4112 - race_output_loss: 3.1624 - age_output_mae: 10.5740 - gender_output_accuracy: 0.5177 - race_output_accuracy: 0.2865 - val_loss: 139.8442 - val_age_output_loss: 137.7639 - val_gender_output_loss: 0.6617 - val_race_output_loss: 1.4186 - val_age_output_mae: 8.7080 - val_gender_output_accuracy: 0.5614 - val_race_output_accuracy: 0.4121
Epoch 2/50
457/457 [==============================] - 71s 155ms/step - loss: 140.2910 - age_output_loss: 138.1059 - gender_output_loss: 0.6843 - race_output_loss: 1.5008 - age_output_mae: 9.1794 - gender_output_accuracy: 0.5604 - race_output_accuracy: 0.3751 - val_loss: 118.6668 - val_age_output_loss: 116.5938 - val_gender_output_loss: 0.6573 - val_race_output_loss: 1.4157 - val_age_output_mae: 8.0196 - val_gender_output_accuracy: 0.5564 - val_race_output_accuracy: 0.4112
Epoch 3/50
457/457 [=========================

In [6]:
def predict_emotion(image):
    # Placeholder for emotion prediction function
    return "happy"

def predict_dress_color(image):
    # Placeholder for dress color prediction function
    return "red"


In [7]:
def custom_predict(image):
    # Preprocess the image
    image = cv2.resize(image, (48, 48))
    image = np.expand_dims(image, axis=0) / 255.0

    # Predict using the model
    age_pred, gender_pred, race_pred = model.predict(image)
    age = int(age_pred[0])
    gender = np.argmax(gender_pred[0])
    race = np.argmax(race_pred[0])

    emotion = predict_emotion(image)
    dress_color = predict_dress_color(image)

    if race == 0:  # Assuming race 0 is Indian
        result = f"Nationality: Indian\nAge: {age}\nEmotion: {emotion}\nDress Color: {dress_color}"
    elif race == 1:  # Assuming race 1 is United States
        result = f"Nationality: United States\nAge: {age}\nEmotion: {emotion}"
    elif race == 2:  # Assuming race 2 is African
        result = f"Nationality: African\nEmotion: {emotion}\nDress Color: {dress_color}"
    else:
        result = f"Nationality: Other\nEmotion: {emotion}"

    return result


In [8]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk

def load_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        image = cv2.imread(file_path)
        result = custom_predict(image)
        result_label.config(text=result)

app = tk.Tk()
app.title("Face Attribute Prediction")

upload_button = tk.Button(app, text="Upload Image", command=load_image)
upload_button.pack()

result_label = tk.Label(app, text="")
result_label.pack()

app.mainloop()


In [ ]:
import tkinter as tk
from tkinter import filedialog, ttk
from PIL import Image, ImageTk
import cv2

# Define a function to load and preprocess the image, then predict the attributes
def load_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        image = cv2.imread(file_path)
        result = custom_predict(image)
        
        # Display the image in the GUI
        image = Image.open(file_path)
        image = image.resize((200, 200))
        image = ImageTk.PhotoImage(image)
        image_label.config(image=image)
        image_label.image = image
        
        result_label.config(text=result)

# Define the main application window
app = tk.Tk()
app.title("Nationality Prediction")

# Use a frame for better layout management
frame = ttk.Frame(app, padding="10")
frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

# Add a title label
title_label = ttk.Label(frame, text="Nationality Prediction", font=("Helvetica", 16))
title_label.grid(row=0, column=0, columnspan=2, pady=(0, 10))

# Add an upload button
upload_button = ttk.Button(frame, text="Upload Image", command=load_image)
upload_button.grid(row=1, column=0, columnspan=2, pady=10)

# Add an image display area
image_label = ttk.Label(frame)
image_label.grid(row=2, column=0, columnspan=2, pady=(0, 10))

# Add a results label
result_label = ttk.Label(frame, text="", wraplength=300)
result_label.grid(row=3, column=0, columnspan=2)

# Configure the grid to expand and fill the window
app.columnconfigure(0, weight=1)
app.rowconfigure(0, weight=1)
frame.columnconfigure(0, weight=1)
frame.columnconfigure(1, weight=1)

app.mainloop()


1/1 [==============================] - 0s 31ms/step


In [ ]:
import streamlit as st
import numpy as np
import cv2
from PIL import Image
from tensorflow.keras.models import load_model

# Load the pre-trained model
model = load_model('nationality_prediction_model.h5')

# Define placeholder functions for emotion and dress color prediction
def predict_emotion(image):
    # Placeholder for emotion prediction function
    return "happy"

def predict_dress_color(image):
    # Placeholder for dress color prediction function
    return "red"

def custom_predict(image):
    # Preprocess the image
    image = cv2.resize(image, (48, 48))
    image = np.expand_dims(image, axis=0) / 255.0

    # Predict using the model
    age_pred, gender_pred, race_pred = model.predict(image)
    age = int(age_pred[0])
    gender = np.argmax(gender_pred[0])
    race = np.argmax(race_pred[0])

    emotion = predict_emotion(image)
    dress_color = predict_dress_color(image)

    if race == 0:  # Assuming race 0 is Indian
        result = f"Nationality: Indian\nAge: {age}\nEmotion: {emotion}\nDress Color: {dress_color}"
    elif race == 1:  # Assuming race 1 is United States
        result = f"Nationality: United States\nAge: {age}\nEmotion: {emotion}"
    elif race == 2:  # Assuming race 2 is African
        result = f"Nationality: African\nEmotion: {emotion}\nDress Color: {dress_color}"
    else:
        result = f"Nationality: Other\nEmotion: {emotion}"

    return result

# Streamlit app
st.title("Nationality Prediction")

uploaded_file = st.file_uploader("Upload an Image", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    # Display the uploaded image
    img = Image.open(uploaded_file)
    st.image(img, caption="Uploaded Image", use_column_width=True)
    
    # Convert to OpenCV format
    img_cv = np.array(img)
    img_cv = cv2
